<a href="https://colab.research.google.com/github/Lucasdz-7/Projeto-Integrador-2/blob/main/C%C3%B3digoAPi2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import pandas as pd
!apt-get install git-lfs -qq
import os
from google.colab import files

In [3]:
# Clonando o repositório do GitHub

repo_url = "https://github.com/Lucasdz-7/Projeto-Integrador-2.git"
repo_dir = "Projeto-Integrador-2"

if not os.path.exists(repo_dir):
    !git clone {repo_url}
    %cd {repo_dir}
else:
    %cd {repo_dir}
    !git pull

/content/Projeto-Integrador-2
Already up to date.


In [4]:
!git lfs install
!git lfs pull

Updated git hooks.
Git LFS initialized.


In [5]:
# Criando Dataframes

csv_path = "Dados Base/dados_ibama.csv"
Dados_IBAMA = pd.read_csv(csv_path, low_memory=False, sep=';', encoding='UTF-8')

In [ ]:
Dados_IBAMA.head(10)

In [6]:
# Filtrando para o período entre 2013 - 2023 e ordenando por ordem crescente

Dados_IBAMA_2013a2023 = Dados_IBAMA[(Dados_IBAMA["Ano"] >= 2013) & (Dados_IBAMA["Ano"] <= 2023)]
Dados_IBAMA_2013a2023 = Dados_IBAMA_2013a2023.sort_values(by='Ano', ascending=True)

In [91]:
Dados_IBAMA_2013a2023.info()

<class 'pandas.core.frame.DataFrame'>
Index: 2941968 entries, 200354 to 281960
Data columns (total 16 columns):
 #   Column                   Dtype 
---  ------                   ----- 
 0   CNPJ                     object
 1   Razão Social             object
 2   Estado                   object
 3   Município                object
 4   Ano                      int64 
 5   Produto                  object
 6   Quantidade Transportada  object
 7   Unidade de Medida        object
 8   Tipo de Transporte       object
 9   Tipo de Armazenamento    object
 10  Plano de Emergência      object
 11  UF - origem              object
 12  Município - origem       object
 13  UF - destino             object
 14  Município - destino      object
 15  Situação Cadastral       object
dtypes: int64(1), object(15)
memory usage: 381.6+ MB


In [ ]:
Dados_IBAMA_2013a2023.head(5)

In [7]:
# Removendo todos outros modais, preservando apenas o rodoviário.

Dados_IBAMA_teste = Dados_IBAMA_2013a2023.query("`Tipo de Transporte` == 'Rodoviário'")

In [8]:
# Removendo zeros e valores nulos.

Dados_IBAMA_teste = Dados_IBAMA_teste[(Dados_IBAMA_teste != 0).all(axis=1)].dropna()
(Dados_IBAMA_teste == 0).sum()

,0
CNPJ,0
Razão Social,0
Estado,0
Município,0
Ano,0
Produto,0
Quantidade Transportada,0
Unidade de Medida,0
Tipo de Transporte,0
Tipo de Armazenamento,0


In [9]:
# Apenas unidades de medidas de kg e litros

Dados_IBAMA_teste = Dados_IBAMA_teste.query(
    "`Unidade de Medida` in ['Grama', 'kilogramas', 'Litro', 'Metro Cúbico', 'Militro', 'Miligrama', 'Tonelada']"
)

In [ ]:
Dados_IBAMA_teste.head(5)

In [ ]:
print(Dados_IBAMA_teste['Unidade de Medida'].unique())

In [ ]:
# 🔸 Lista de palavras-chave para filtrar produtos perigosos ou especiais
# Inclui:
# - Combustíveis e derivados (ex: gasolina, diesel, etanol, GLP, querosene, óleo, lubrificantes)
# - Substâncias químicas perigosas (ex: ácidos, nitratos, solventes, cloro, amônia, formaldeído, etc.)
# - Termos relacionados a risco (inflamável, tóxico, corrosivo, oxidante, radioativo, explosivo...)
# - Resíduos e efluentes industriais (lama, lodo, borra, emulsão, catalisadores, defensivos, fertilizantes químicos)
# - Gases e criogênicos (oxigênio, nitrogênio, CO2, gás comprimido, criogênico...)
# - Metais pesados e compostos correlatos (mercúrio, chumbo, dicromato, etc.)
# - Produtos industriais diversos (tintas, vernizes, betume, adesivos químicos, aditivos, borracha sintética...)

palavras_perigosas = [
    'gasolina', 'diesel', 'etanol', 'álcool', 'biodiesel', 'combustível',
    'querosene', 'gnv', 'gás liquefeito', 'gás natural', 'butano', 'propano',
    'glp', 'naphta', 'nafta', 'óleo', 'lubrificante',

    'ácido', 'base', 'alcalino', 'hidróxido', 'nitrato', 'peróxido',
    'hipoclorito', 'cloro', 'amônia', 'soda cáustica', 'cloreto', 'sulfato',
    'fosfato', 'ácetico', 'acético', 'cianeto', 'anidro', 'solvente',
    'tolueno', 'xileno', 'benzeno', 'acetona', 'formaldeído', 'fenol',
    'butil', 'metanol', 'isopropanol', 'hexano', 'heptano', 'éter',

    'inflamável', 'tóxico', 'corrosivo', 'explosivo', 'oxidante',
    'radioativo', 'perigoso', 'reativo', 'irritante', 'comburente',
    'poluente', 'resíduo perigoso', 'resíduo químico',

    'resíduo', 'lama', 'lodo', 'efluente', 'emulsão', 'borra', 'bagaço químico',
    'catalisador', 'pesticida', 'herbicida', 'inseticida', 'defensivo',
    'fertilizante químico', 'corante industrial', 'pigmento químico',

    'oxigênio', 'nitrogênio', 'hélio', 'argônio', 'co2', 'co', 'freon',
    'refrigerante', 'criogenico', 'criogênico', 'gás comprimido', 'gás liquefeito',

    'mercúrio', 'chumbo', 'cádmio', 'arsênio', 'cobre', 'níquel',
    'zinco', 'manganês', 'crômico', 'cromato', 'dicromato',

    'betume', 'asfalto', 'tinta', 'verniz', 'cola industrial',
    'adesivo químico', 'detergente industrial', 'espessante químico',
    'aditivo químico', 'plástico fundido', 'borracha sintética'
]

mask = Dados_IBAMA_teste['Produto'].str.lower().str.contains('|'.join(palavras_perigosas), na=False)
Dados_IBAMA_teste = Dados_IBAMA_teste[mask]


In [ ]:
contagem_valores = Dados_IBAMA_teste['Produto'].value_counts()
print(contagem_valores)


In [ ]:
Dados_IBAMA_teste['Produto'].value_counts().tail(50)


In [ ]:

# Quantidade original e unidade normalizada
qtd = Dados_IBAMA_teste['Quantidade Transportada'].fillna(0)
unidade = Dados_IBAMA_teste['Unidade de Medida'].fillna('').str.strip().str.lower()

# Cria colunas novas zeradas
Dados_IBAMA_teste['qntd_Kg'] = 0.0
Dados_IBAMA_teste['qntd_Litros'] = 0.0

# ---- Conversão para kg ----
mask_kg = unidade.isin(['tonelada', 'kilogramas', 'grama', 'miligrama'])
Dados_IBAMA_teste.loc[mask_kg & (unidade == 'tonelada'), 'qntd_Kg'] = qtd * 1000
Dados_IBAMA_teste.loc[mask_kg & (unidade == 'kilogramas'), 'qntd_Kg'] = qtd
Dados_IBAMA_teste.loc[mask_kg & (unidade == 'grama'), 'qntd_Kg'] = qtd / 1000
Dados_IBAMA_teste.loc[mask_kg & (unidade == 'miligrama'), 'qntd_Kg'] = qtd / 1_000_000

# ---- Conversão para litros ----
mask_litro = unidade.isin(['litro', 'militro', 'metro cúbico'])
Dados_IBAMA_teste.loc[mask_litro & (unidade == 'litro'), 'qntd_Litros'] = qtd
Dados_IBAMA_teste.loc[mask_litro & (unidade == 'militro'), 'qntd_Litros'] = qtd / 1000
Dados_IBAMA_teste.loc[mask_litro & (unidade == 'metro cúbico'), 'qntd_Litros'] = qtd * 1000

# Agora, para linhas que não são kg nem litro, as duas colunas ficam 0
# Se quiser, você pode colocar np.nan em vez de 0
Dados_IBAMA_teste.loc[~(mask_kg | mask_litro), ['qntd_Kg','qntd_Litros']] = np.nan

# Visualizar resultado
print(Dados_IBAMA_teste[['Quantidade Transportada','Unidade de Medida','qntd_Kg','qntd_Litros']].head(10))


In [ ]:
Dados_IBAMA_teste.head(10)